In [ ]:
import os
import numpy as np
import pandas as pd
import requests
import xlsxwriter 
import math
import yfinance as yf
from dotenv import load_dotenv
# import requests
import requests_cache
import requests_ratelimiter
import pyrate_limiter


session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
# ticker = yf.Ticker('AAPL', session=session)
# # The scraped response will be stored in the cache
# ticker.actions

# class CachedLimiterSession(
#     requests_cache.session.CacheMixin,
#     requests_ratelimiter.LimiterMixin,
#     requests.Session
#     ):
#     pass

# session = CachedLimiterSession(
#     limiter=pyrate_limiter.Limiter(
#         pyrate_limiter.RequestRate(
#             2, pyrate_limiter.Duration.SECOND*5
#             ),  # max 2 requests per 5 seconds
#     bucket_class=requests_ratelimiter.MemoryQueueBucket,
#     backend=requests_cache.SQLiteCache("yfinance.cache"),
#     )
# )


In [ ]:
import datetime as dt
import os

In [ ]:
stocks = pd.read_csv('2023-06-01-sp500.csv')
load_dotenv()

In [ ]:
stocks

In [ ]:
symbol = "AAPL"
currentStock = yf.Ticker(f"{symbol}", session=session)
currentStock.info

In [ ]:
currentStock.actions

In [ ]:
currentStock.info['currentPrice']

In [ ]:
currentStock.info['marketCap']

In [ ]:
headings = [ 'Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to Buy']
result = pd.DataFrame(columns=headings)

result

In [ ]:
def chunks(list, n):
    """Yield successive n-sized chunks from list"""
    for i in range(0, len(list), n):
        yield list[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 101))

symbol_groups

In [ ]:
every_str = []
for i in range(0, len(symbol_groups)):
    every_str.append(','.join(symbol_groups[i]))
every_str

In [ ]:
current_values = pd.DataFrame(columns=headings)
for symbol_string in every_str:
    for stock in symbol_string.split(','):
        this_stock = yf.Ticker(f"{stock}", session=session)
        try:
            current_values = current_values.append(
                pd.Series(
                [
                    stock,
                    this_stock.info['currentPrice'],
                    this_stock.info['marketCap'],
                    "N/A"
                ],
                index = headings
                ),
                ignore_index=True
        )
        except(KeyError):
            pass
current_values

In [ ]:
current_values

In [ ]:
today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')    
current_values.to_csv(f'{today}-full_sp500_data.csv')

In [ ]:
full_list = list(stocks['Ticker'])
current_values = pd.DataFrame(columns=headings)
for stock in full_list:
    this_stock = yf.Ticker(f"{stock}", session=session)
    current_values = current_values.append(
    pd.Series(
    [
        stock,
        this_stock.info['currentPrice'],
        this_stock.info['marketCap'],
        "N/A"
    ],
    index = headings
    ),
    ignore_index=True
)

current_values

In [ ]:
current_values

In [ ]:
aapl = yf.Ticker('AAPL', session=session)

aapl.info

In [ ]:
new_columns = [*aapl.info.keys()]
new_columns #at some points taake all of these keys and put into csv

In [ ]:
[*aapl.info.values()]

In [ ]:
aapl_df = pd.DataFrame(columns=new_columns)
aapl_df = aapl_df.append(
    pd.Series(
        [*aapl.info.values()],
        index = headings
        ),
        ignore_index=True
    )

aapl_df

In [ ]:
val = None

while val == None:
    portfolio_size = input('Enter the value of your portfolio: ')
    try:
        val = float(portfolio_size)
    except(ValueError):
        print('Not a number, please enter a legitamate value:')

val

In [ ]:

today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
current_values = pd.read_csv(f'{today}-full_sp500_data.csv')

In [ ]:
current_values = pd.read_csv('2023-06-01-15-54-22-full_sp500_data.csv')

current_values = current_values.drop('Unnamed: 0', axis=1)

position_size = val/len(current_values.index)
for i in range(0, len(current_values.index)):
    current_values.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / current_values['Stock Price'][i])

current_values

In [ ]:
file_path='.\{today}-recommended_trades.xlsx'

writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
current_values.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:
__file__

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()